In [2]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
from arcgis.features import FeatureLayer
import re


# Make Feature Layer

In [3]:
sdfCombinedForecasts = pd.DataFrame.spatial.from_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp')

In [4]:
sdfCombinedForecasts = sdfCombinedForecasts[sdfCombinedForecasts['ROUTE']!='UTA']
sdfCombinedForecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0023_000.0,0023,0.000,1.564,1.585102,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422233.4001000002, 4607564.0001],..."
1,1,0,0023_001.6,0023,1.564,2.146,0.580084,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422182.7999999998, 4610094.1], [4..."
2,2,0,0023_002.1,0023,2.146,2.533,0.384494,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422217, 4611027], [422220.4000000..."
3,3,0,0023_002.5,0023,2.533,4.170,1.633041,5.0,2.0,Cache,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[421790.7599999998, 4611304.15], [..."
4,4,0,0023_004.2,0023,4.170,6.289,2.114815,5.0,2.0,UDOT,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[420610.13999999966, 4613651.36999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,5833,0,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5.0,1.0245,0.0,"{""paths"": [[[424308.5, 4489341.199999999], [42..."
5834,5834,0,WFRC_8473,WFRC,0.000,0.000,0.714015,11.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[417141.7167999996, 4545904.8543],..."
5835,5835,0,WFRC_8474,WFRC,0.000,0.000,0.232025,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422745.79000000004, 4480897.97000..."
5836,5836,0,WFRC_8475,WFRC,0.000,0.000,0.587918,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422006.9967, 4481267.654200001], ..."


In [5]:
#already done in previous notebook
#sdfCombinedForecasts.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

# Create Route Json

In [6]:
# Route Json
# R: Route Label (string, first 4 of SEGID)
# C: CO_FIPS (integer)
# label: Prefix of 
#        "I-"  + Route Label with 0s removed : for interstates
#        "Hwy" + Route Label with 0s removed : us and state routes
#        "FA"  + Route Label with 0s removed : federal aid routes
#        "WFRC/MAG/UDOT/UTA" 


dfRoutePrefix = pd.DataFrame([
    [   1,   14, 'Hwy'],
    [  15,   15, 'I-' ],
    [  16,   69, 'Hwy'],
    [  70,   70, 'I-' ],
    [  71,   79, 'Hwy'],
    [  80,   80, 'I-' ],
    [  81,   83, 'Hwy'],
    [  84,   84, 'I-' ],
    [  85,  214, 'Hwy'],
    [ 215,  215, 'I-' ],
    [ 216,  999, 'Hwy'],
    [1000, 5000, 'FA' ]
], columns=('routeFrom','routeTo','prefix'))
dfRoutePrefix

,routeFrom,routeTo,prefix
0,1,14,Hwy
1,15,15,I-
2,16,69,Hwy
3,70,70,I-
4,71,79,Hwy
5,80,80,I-
6,81,83,Hwy
7,84,84,I-
8,85,214,Hwy
9,215,215,I-


In [7]:
dfRoutes = sdfCombinedForecasts.groupby(['ROUTE','CO_FIPS'],as_index=False).agg(numSeg=('SEGID','count'))
dfRoutes['CO_FIPS'  ] = dfRoutes['CO_FIPS'].astype(int)
dfRoutes['ROUTE_numeric'] = pd.to_numeric(dfRoutes['ROUTE'], errors='coerce')

dfRoutes

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric
0,0006,49,36,6.0
1,0013,3,10,13.0
2,0015,3,5,15.0
3,0015,11,19,15.0
4,0015,35,27,15.0
...,...,...,...,...
827,MAG,49,792,NaN
828,WFRC,3,24,NaN
829,WFRC,11,47,NaN
830,WFRC,35,303,NaN


In [8]:
# Step 1: Add a dummy key to both DataFrames
dfRoutes['key'] = 1
dfRoutePrefix['key'] = 1

# Step 2: Merge on the dummy key
merged_df = pd.merge(dfRoutes, dfRoutePrefix, on='key')

# Step 3: Filter rows where ROUTE_numeric falls within the routeFrom and routeTo range
# Modify the filter to preserve NaN values (Step 3)
# We use | (OR) to add a condition that explicitly checks for NaNs in ROUTE_numeric
dfRoutePrefixWithPrefix = merged_df[((merged_df['ROUTE_numeric'] >= merged_df['routeFrom']) & 
                                     (merged_df['ROUTE_numeric'] <= merged_df['routeTo'])) |
                                     (merged_df['ROUTE_numeric'].isna())].copy()

# Fill specific columns with '' where ROUTE_numeric is NaN
# Adjust 'your_column_name_here' to the actual column(s) you want to fill for NaN ROUTE_numeric rows
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'prefix'   ] = dfRoutePrefixWithPrefix['ROUTE']
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeFrom'] = 0
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeTo'  ] = 0

dfRoutePrefixWithPrefix['ROUTE_numeric'].fillna(0, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop_duplicates()

# Step 4: Drop the dummy key column
dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop(columns=['key'])
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix
0,0006,49,36,6.0,1,14,Hwy
12,0013,3,10,13.0,1,14,Hwy
25,0015,3,5,15.0,15,15,I-
37,0015,11,19,15.0,15,15,I-
49,0015,35,27,15.0,15,15,I-
...,...,...,...,...,...,...,...
9924,MAG,49,792,0.0,0,0,MAG
9936,WFRC,3,24,0.0,0,0,WFRC
9948,WFRC,11,47,0.0,0,0,WFRC
9960,WFRC,35,303,0.0,0,0,WFRC


In [9]:
dfRoutePrefixWithPrefix['ROUTE_text'] = dfRoutePrefixWithPrefix['ROUTE_numeric'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)
dfRoutePrefixWithPrefix.loc[(dfRoutePrefixWithPrefix['ROUTE_text']=="0"), 'ROUTE_text'] = ''
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix,ROUTE_text
0,0006,49,36,6.0,1,14,Hwy,6
12,0013,3,10,13.0,1,14,Hwy,13
25,0015,3,5,15.0,15,15,I-,15
37,0015,11,19,15.0,15,15,I-,15
49,0015,35,27,15.0,15,15,I-,15
...,...,...,...,...,...,...,...,...
9924,MAG,49,792,0.0,0,0,MAG,
9936,WFRC,3,24,0.0,0,0,WFRC,
9948,WFRC,11,47,0.0,0,0,WFRC,
9960,WFRC,35,303,0.0,0,0,WFRC,


In [10]:
dfRoutePrefixWithPrefix.fillna('',inplace=True)
dfRoutePrefixWithPrefix['label']=dfRoutePrefixWithPrefix['prefix'] + dfRoutePrefixWithPrefix['ROUTE_text']

dfRoutePrefixWithPrefix.rename(columns={'ROUTE':'R','CO_FIPS':'C'}, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix[['R','C','label']]

dfRoutePrefixWithPrefix

,R,C,label
0,0006,49,Hwy6
12,0013,3,Hwy13
25,0015,3,I-15
37,0015,11,I-15
49,0015,35,I-15
...,...,...,...
9924,MAG,49,MAG
9936,WFRC,3,WFRC
9948,WFRC,11,WFRC
9960,WFRC,35,WFRC


In [11]:
# export to JSON
dfRoutePrefixWithPrefix.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/routes.json', orient='records', indent=4)

# Create Forecasts json

In [12]:
sdfCombinedForecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0023_000.0,0023,0.000,1.564,1.585102,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422233.4001000002, 4607564.0001],..."
1,1,0,0023_001.6,0023,1.564,2.146,0.580084,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422182.7999999998, 4610094.1], [4..."
2,2,0,0023_002.1,0023,2.146,2.533,0.384494,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422217, 4611027], [422220.4000000..."
3,3,0,0023_002.5,0023,2.533,4.170,1.633041,5.0,2.0,Cache,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[421790.7599999998, 4611304.15], [..."
4,4,0,0023_004.2,0023,4.170,6.289,2.114815,5.0,2.0,UDOT,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[420610.13999999966, 4613651.36999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,5833,0,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5.0,1.0245,0.0,"{""paths"": [[[424308.5, 4489341.199999999], [42..."
5834,5834,0,WFRC_8473,WFRC,0.000,0.000,0.714015,11.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[417141.7167999996, 4545904.8543],..."
5835,5835,0,WFRC_8474,WFRC,0.000,0.000,0.232025,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422745.79000000004, 4480897.97000..."
5836,5836,0,WFRC_8475,WFRC,0.000,0.000,0.587918,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422006.9967, 4481267.654200001], ..."


In [13]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^F\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 5]

# Display the list of columns
print(columns_matching_pattern)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justneededcols = sdfCombinedForecasts[['SEGID','CO_FIPS'] + columns_matching_pattern]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justneededcols

['F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050']


,SEGID,CO_FIPS,F2019,F2023,F2028,F2032,F2042,F2050
0,0023_000.0,5.0,1969.0,2000.0,2400.0,2600.0,2700.0,3100.0
1,0023_001.6,5.0,2955.0,2900.0,3200.0,3400.0,3300.0,3500.0
2,0023_002.1,5.0,2955.0,2900.0,3400.0,3500.0,3600.0,4100.0
3,0023_002.5,5.0,2209.0,2200.0,2600.0,2700.0,2700.0,3200.0
4,0023_004.2,5.0,2209.0,2200.0,2500.0,2700.0,2700.0,3100.0
...,...,...,...,...,...,...,...,...
5833,WFRC_8472,35.0,0.0,0.0,0.0,0.0,0.0,0.0
5834,WFRC_8473,11.0,0.0,6900.0,6400.0,6800.0,7500.0,8300.0
5835,WFRC_8474,35.0,0.0,0.0,0.0,0.0,0.0,0.0
5836,WFRC_8475,35.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justneededcols, id_vars=[col for col in sdfCombinedForecasts_justneededcols.columns if col not in columns_matching_pattern], 
                    value_vars=columns_matching_pattern, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[1:]).astype(int)

melted_df['F'] = melted_df['Value'].astype(int)

melted_df['CO_FIPS'] = melted_df['CO_FIPS'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justneededcols_melted = melted_df[['S','Y','F','CO_FIPS']]
sdfCombinedForecasts_justneededcols_melted

,S,Y,F,CO_FIPS
0,0023_000.0,2019,1969,5
1,0023_001.6,2019,2955,5
2,0023_002.1,2019,2955,5
3,0023_002.5,2019,2209,5
4,0023_004.2,2019,2209,5
...,...,...,...,...
34489,WFRC_8472,2050,0,35
34490,WFRC_8473,2050,8300,11
34491,WFRC_8474,2050,0,35
34492,WFRC_8475,2050,0,35


In [15]:
# export to JSON
sdfCombinedForecasts_justneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/forecasts.json', orient='records', indent=4)

# Create Observed

In [16]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^AADT\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern_AADT = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 8]

# Display the list of columns
print(columns_matching_pattern_AADT)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justAADTneededcols = sdfCombinedForecasts[['SEGID'] + columns_matching_pattern_AADT]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justAADTneededcols

['AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981']


,SEGID,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,...,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981
0,0023_000.0,0.0,0.0,0.0,1969.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0023_001.6,0.0,0.0,0.0,2955.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0023_002.1,0.0,0.0,0.0,2955.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0023_002.5,0.0,0.0,0.0,2209.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0023_004.2,0.0,0.0,0.0,2209.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5834,WFRC_8473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5835,WFRC_8474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5836,WFRC_8475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
sdfCombinedForecasts_justAADTneededcols.columns

Index(['SEGID', 'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018',
       'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012',
       'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006',
       'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000',
       'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994',
       'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988',
       'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982',
       'AADT1981'],
      dtype='object')

In [18]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justAADTneededcols, id_vars=[col for col in sdfCombinedForecasts_justAADTneededcols.columns if col not in columns_matching_pattern_AADT], 
                    value_vars=columns_matching_pattern_AADT, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[4:]).astype(int)

melted_df = melted_df[melted_df['Year']>=2000]

melted_df['O'] = melted_df['Value'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justAADTneededcols_melted = melted_df[['S','Y','O']]
sdfCombinedForecasts_justAADTneededcols_melted

,S,Y,O
0,0023_000.0,2022,0
1,0023_001.6,2022,0
2,0023_002.1,2022,0
3,0023_002.5,2022,0
4,0023_004.2,2022,0
...,...,...,...
132222,WFRC_8472,2000,0
132223,WFRC_8473,2000,0
132224,WFRC_8474,2000,0
132225,WFRC_8475,2000,0


In [19]:
# export to JSON
sdfCombinedForecasts_justAADTneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/observed.json', orient='records', indent=0)

# create segments json

In [20]:
dfSegments = pd.merge(sdfCombinedForecasts[['SEGID','ROUTE','BMP']],dfRoutePrefixWithPrefix, left_on='ROUTE', right_on='R')
dfSegments.rename(columns={'SEGID':'S','label':'N'},inplace=True)

# set label
dfSegments['L'] = dfSegments['N'] + ' ~MP' + dfSegments['BMP'].round(1).astype(str)
dfSegments.loc[(dfSegments['R'].isin(['WFRC','MAG','CACHE','DIXIE','UDOT','UTA'])), 'L'] = dfSegments['S']


dfSegments.drop(columns=['ROUTE','BMP'],inplace=True)
dfSegments

,S,R,C,N,L
0,0023_000.0,0023,5,Hwy23,Hwy23 ~MP0.0
1,0023_001.6,0023,5,Hwy23,Hwy23 ~MP1.6
2,0023_002.1,0023,5,Hwy23,Hwy23 ~MP2.1
3,0023_002.5,0023,5,Hwy23,Hwy23 ~MP2.5
4,0023_004.2,0023,5,Hwy23,Hwy23 ~MP4.2
...,...,...,...,...,...
9598,WFRC_8475,WFRC,57,WFRC,WFRC_8475
9599,WFRC_8476,WFRC,3,WFRC,WFRC_8476
9600,WFRC_8476,WFRC,11,WFRC,WFRC_8476
9601,WFRC_8476,WFRC,35,WFRC,WFRC_8476


In [21]:
# export to JSON
dfSegments.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/segments.json', orient='records', indent=0)